In [ ]:
#Las cosas importantes porque son importadas

import cv2  
import math 

from ultralytics import YOLO

from collections import defaultdict
import numpy as np

import cv2
import pytesseract
from pytesseract import Output

model = YOLO('yolo11n.pt') #Contenedores

filename = "Parking.mp4"
results = model(filename, show=True)

cv2.destroyAllWindows()